
    
    
## <H2 align="center"> <font color='white'> **MAF1731B**: Microstructure and Trading Systems </H2>

### <H3 align="center"> <font color='white'> **Laboratory 4**: Market Microstructure </H3>

 <H5 align="center"> <em> Professor: Villalobos Ramos Omar Antonio <em>

 <p align="center"> <img style=" margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="350" height="150" /> 

 <strong> <H5 align="left"> Student:
Esteban Márquez Delgado <strong>

 *Registry*:
if700637: https://github.com/EstebanMqz/MyST_LAB_4

 <center> <font color= #555555> <font size = 4> November 2022 </a> <font color= #555555> <font size = 4> </a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Laboratory 4 </font> <br> <br> <font color= #555555> <font size = 5> Market Microstructure  </font>

<hr style="border:0.02in solid gray"> </hr>

### <font color= #555555> <font size = 4> Abstract </font>

This document was prepared as delivery for Laboratory 4 of Microstructure and Trading Systems - MAF1731B class during the Autumn 2022 course at ITESO for the Bachelor of Financial Engineering. In order to analyze the market microstructure in criptocurrencies markets containing the following 3 sections:

1. **Consume CCXT data**: Download Criptocurrencies Order Books from exchanges (kraken, ftx, currencycom, coinmate, etc).

2. **Microstructure Visualization**: Build Time Series using the Order Books obtained in the previous point from the following elements of the Market Microstructure for each asset and exchange (Levels, Bid Volume, Ask Volume, Total Volume, Mid Price, VWAP.)

3. **Microstructure Modeling**: Estimation of Roll Effective Spread using the lagged series at t-5.



<font color= lightblue> <font size = 4> 0. Libraries and Dependencies: </font>

<font color= gray>

**Note**: To run this notebook it is necessary to have the following libraries contained in the [requirements.txt](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/requirements.txt) script of this project.

To install the libraries in the script run the following cell:

In [1]:
!pip install -r requirements.txt

ERROR: Invalid requirement: '"""' (from line 1 of requirements.txt)


If you prefer, manual installation can be done with the individual installation of the following libraries:

<font color= lightblue> <font size = 1> 
+ pandas >= 1.3.4
+ numpy >= 1.19.1
+ jupyter >= 1.0.0
+ chart_studio >= 1.1
+ plotly >= 4.14
+ logging >= 3.11
+ datetime >= 4.7
+ pandas_datareader >= 0.10.0
+ tk >= 0.10
+ random >= 3.11
+ plotly >= 5.60
+ yahoofinancials >= 1.60
+ scipy >= 1.7.3
+ regex >= 2022.3.15
+ matplotlib >= 3.5.1
+ fire >= 0.4.0
+ ccxt >= 2.1.33
+ time >= 3.11 


### <font color= lightblue> <font size = 4> 0.1 Introduction: </font>
In this laboratory we will use information from the Cryptocurrency Order Book of various international exchanges, with the primary objective of visualizing and modeling the microstructure of the market through the behavior of the Order Book and its characteristics such as; Matching Engine, Levels, Ask-Bid Volumes, etc.
<font color= gray>




##### *General help:*
<font color= lightgreen> **Note**: Progress checks at the end of each section are highlighted. <font color= white>

In order, to retrieve specific help from the scripts tree in the project execute the help function on the script or in specific modules from the following:
+ [data.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/data.py)
+ [functions.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/functions.py)
+ [visualizations.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/visualizations.py)

In [2]:
## Libraries
import pandas as pd # Analysis and data management
import pandas_datareader as pdr
import pandas_datareader.data as web #Data extraction from the web
import numpy as np
from datetime import datetime, timedelta, date
from tkinter.ttk import Style
from tkinter import Y
import random as rand
import plotly.graph_objects as go #Plotly
import plotly.express as px
import warnings
from yahoofinancials import YahooFinancials # Yfinance 
import yfinance as yf
import ccxt #Criptocurrencies
import logging as log #Exchanges 
from scipy.optimize import minimize # Optimization 
from collections import Counter #Counter
import time
import re
import io 
import glob
import os
#.Py archives in repo
import functions as fn 
import visualizations as vs
import data as dt
from os import path
import fire
%matplotlib inline

<font color= orange> <font size = 5> 1. Consume CCXT data: </font>

In order to know the exchanges avaliable in CCTX module in Python for Criptocurrencies data download, the following getattr function *find_exchanges* is executed:

In [3]:
help(fn.find_exchanges)

Help on function find_exchanges in module functions:

find_exchanges(features=None, is_authenticated=False)
    Function that returns avaliable cryptocurrencies exchanges in Python for CCTX module.
    
        Parameters:
        ----------
        features: None 
        is_authenticated: None 
    
        Returns:
        -------
        exchange_names: Array of authenticated exchanges avaliable in CCTX.



From all the 120 exchanges avaliable we pick **binance**, **ftx** and **bytetrade** because of their fast fetching.

In [4]:
#Available exchanges in CCTX
fn.find_exchanges()[:] #len 120

['aax',
 'alpaca',
 'ascendex',
 'bequant',
 'bibox',
 'bigone',
 'binance',
 'binancecoinm',
 'binanceus',
 'binanceusdm',
 'bit2c',
 'bitbank',
 'bitbay',
 'bitbns',
 'bitcoincom',
 'bitfinex',
 'bitfinex2',
 'bitflyer',
 'bitforex',
 'bitget',
 'bithumb',
 'bitmart',
 'bitmex',
 'bitopro',
 'bitpanda',
 'bitrue',
 'bitso',
 'bitstamp',
 'bitstamp1',
 'bittrex',
 'bitvavo',
 'bkex',
 'bl3p',
 'blockchaincom',
 'btcalpha',
 'btcbox',
 'btcex',
 'btcmarkets',
 'btctradeua',
 'btcturk',
 'buda',
 'bw',
 'bybit',
 'bytetrade',
 'cex',
 'coinbase',
 'coinbaseprime',
 'coinbasepro',
 'coincheck',
 'coinex',
 'coinfalcon',
 'coinmate',
 'coinone',
 'coinspot',
 'crex24',
 'cryptocom',
 'currencycom',
 'delta',
 'deribit',
 'digifinex',
 'exmo',
 'flowbtc',
 'fmfwio',
 'ftx',
 'ftxus',
 'gate',
 'gateio',
 'gemini',
 'hitbtc',
 'hitbtc3',
 'hollaex',
 'huobi',
 'huobijp',
 'huobipro',
 'idex',
 'independentreserve',
 'indodax',
 'itbit',
 'kraken',
 'kucoin',
 'kucoinfutures',
 'kuna',
 'lat

Now that we have selected the exchanges, we pick random criptocurrencies and verify that they are all avaliable in their platform.

In [5]:
exchanges= ['binance','ftx', 'bytetrade'] #Selected exchanges from list (can't modify because of fn and vs .py)
criptos = ['BTC/USDT','ETH/USDT','XRP/USDT'] #Criptos OBs (modify tickers here)

<font color= lightblue> <font size = 4> Order Book Levels & OHLCV for 3 exchanges and Criptos: </font>

Now we proceed to download data from exchanges for all criptocurrencies.

In [6]:
help(vs.cctx_download)

Help on function cctx_download in module visualizations:

cctx_download(lvls, cripto, exchange)
    Function that returns prices and quantities of given levels (as integer) Bids & Asks in an Orderbook
    for the specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and
    from the following exchanges: binance, ftx or ascendex (as string).
    
        Parameters:
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable ticker (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        Returns:
        -------
        levels_ob_bid: Prices and Quantities of Bids (pos [0]) and Asks (pos [1]) as dataframes.



We get n=25 levels for each OB, and proceed to download combinations from criptos and exchanges from their lists [0:2] (9).

We will start with binance and BTC/USDT.

In [7]:
params= 25, criptos[0], exchanges[0]
params

(25, 'BTC/USDT', 'binance')

+ <font color= lightgray> <font size = 2> *Bids*: </font>

In [8]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,18581.78,0.01226
1,18581.68,0.00336
2,18581.63,0.02000
3,18581.45,0.04000
4,18581.28,0.00483


+ <font color= lightgray> <font size = 2> *Asks*: </font>

In [9]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,18583.07,0.11594
1,18583.80,0.00688
2,18583.87,0.03229
3,18583.96,0.02818
4,18584.38,0.00694


In [10]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-08T19:00:02.785Z,18922.9,18929.9,18829.6,18858.96,1183.22717


We plot the results for the first combination of ticker and exchanges with the visualization *OBLvls*.

In [11]:
help(vs.OBLvls_hist)

Help on function OBLvls_hist in module visualizations:

OBLvls_hist(lvls, cripto, exchange)
    Function that plots an horizontal histogram in plotly for CriptoCurrencies OB/
    
        Parameters
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable ticker (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        x: Quantity (col) of the given cripto (str) for given lvls (int) in Order Book.
        y: Prices (col) of the given cripto (str) for given lvls (int) in Order Book.
    
        Returns
        -------
        Histogram of Order Book prices and quantities for n given lvls.



In [12]:
vs.OBLvls_hist(params[0],params[1], params[2])

We will proceed for binance and ETH/USDT.

In [13]:
params= 25, criptos[1], exchanges[0]
params

(25, 'ETH/USDT', 'binance')

In [14]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,1342.04,2.5012
1,1342.02,1.0000
2,1342.01,1.0000
3,1342.00,52.9194
4,1341.99,0.2174


In [15]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,1342.05,0.2426
1,1342.12,8.9386
2,1342.22,0.3234
3,1342.23,3.1718
4,1342.26,8.2601


In [16]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-08T19:00:06.670Z,1386.43,1386.9,1374.17,1378.45,4121.8351


In [17]:
vs.OBLvls_hist(params[0],params[1], params[2])

We continue with the exchange binance and XRP/USDT.

In [18]:
params= 25, criptos[2], exchanges[0]
params

(25, 'XRP/USDT', 'binance')

In [19]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,0.4053,9777.0
1,0.4052,8000.0
2,0.4051,11751.0
3,0.4050,4154.0
4,0.4048,751.0


In [20]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,0.4058,96984.0
1,0.4059,8883.0
2,0.4060,78234.0
3,0.4061,2309.0
4,0.4062,43312.0


In [21]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-08T19:00:10.654Z,0.4141,0.4142,0.4117,0.4125,1817854.0


In [22]:
vs.OBLvls_hist(params[0],params[1], params[2])

We could retrieve criptocurrencies with any of the 3 exchanges declared in our list *exchanges* but to avoid being repetitive we will make one more random example and we will conclude with a dataframe. For section section **2.** *Microstructure Visualization* a dataframe with different cols and plots that allow us to compare differences between exchanges will be created.

In [23]:
params= 25, criptos[rand.randint(0,2)], exchanges[rand.randint(0,2)]
params

(25, 'ETH/USDT', 'binance')

In [24]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])

In [25]:
vs.OBLvls_hist(params[0],params[1], params[2])

In [26]:
v_params0= 25, criptos[0], exchanges[0]
v_params1= 25, criptos[0], exchanges[1]
v_params2= 25, criptos[0], exchanges[2]
v_params0, v_params1, v_params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

In [27]:
n=10
verif_0 = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1 = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2 = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In [33]:
help(vs.verif_ex1)

Help on function verif_ex1 in module visualizations:

verif_ex1(lvls, cripto, exchange, n)
    Function that returns Verif 1 for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable ticker (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    
    Returns:
    -------
    df1 = Section 1 Verification.



<strong> 
<font color= lightgreen> <font size = 3> Progress check </font>

Some differences between criptocurrencies within different exchanges can be appreciated. Now we can realize which exchange can provide us with the lowest spread and best liquidity. Clearly, prices are the most important factor for investments. 

<font color= lightblue> <font size = 2> Verification </font>

In [32]:
verif_df = fn.concatenate(verif_0[0], verif_1[0], verif_2[0], n)
verif_df

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-08T19:00:17.509,18644.73,18642.95,2.43243,3.930370,1.78,18642.09
1,binance,2022-11-08T19:00:19.968,18641.59,18640.43,5.49092,47.371870,1.16,18642.93
2,binance,2022-11-08T19:00:22.504,18658.42,18657.16,2.75723,4.512060,1.26,18659.01
3,binance,2022-11-08T19:00:24.974,18681.75,18679.93,0.55864,2.927850,1.82,18680.43
4,binance,2022-11-08T19:00:27.517,18686.07,18684.36,1.75597,3.012220,1.71,18682.59
5,binance,2022-11-08T19:00:29.938,18687.16,18687.03,2.23197,1.316340,0.13,18686.88
6,binance,2022-11-08T19:00:33.268,18681.70,18677.35,2.61967,1.977200,4.35,18679.38
7,binance,2022-11-08T19:00:35.777,18699.87,18694.68,26.47913,2.668320,5.19,18699.87
8,binance,2022-11-08T19:00:38.162,18695.94,18692.59,26.71302,1.389890,3.35,18694.20
9,binance,2022-11-08T19:00:40.562,18714.97,18712.45,1.07039,4.002780,2.52,18714.98


<font color= lightblue> <font size = 2> OHLCV </font>

For each timeframe a candle is formed and it illustrates how much Volatility every timestamp shows within exchanges and this is important to know in algorithmic trading. Not to mention the fact that stock exchanges do not provide candle data in ms for many reasons.

In [31]:
verif_df = fn.concatenate(verif_0[1], verif_1[1], verif_2[1], n)
verif_df

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-08T19:00:17.509,18600.51,18646.86,18557.51,18642.09,508.54262
1,binance,2022-11-08T19:00:19.968,18600.51,18651.35,18557.51,18642.93,558.92230
2,binance,2022-11-08T19:00:22.504,18600.51,18663.98,18557.51,18659.01,658.26441
3,binance,2022-11-08T19:00:24.974,18600.51,18680.98,18557.51,18680.43,741.33661
4,binance,2022-11-08T19:00:27.517,18600.51,18693.08,18557.51,18682.59,791.59869
5,binance,2022-11-08T19:00:29.938,18600.51,18694.28,18557.51,18686.88,840.58573
6,binance,2022-11-08T19:00:33.268,18600.51,18696.97,18557.51,18679.38,925.29398
7,binance,2022-11-08T19:00:35.777,18600.51,18700.00,18557.51,18699.87,958.32888
8,binance,2022-11-08T19:00:38.162,18600.51,18700.00,18557.51,18694.20,991.05049
9,binance,2022-11-08T19:00:40.562,18600.51,18714.98,18557.51,18714.98,1054.61667


<font color= orange> <font size = 5> 2. Microstructure Visualization </font>

In [34]:
help(vs.Micro)

Help on function Micro in module visualizations:

Micro(lvls, cripto, exchange, n)
    Function that returns Microstructure OB and OHLCV for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable ticker (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    
    Returns:
    -------
    Micro[0]: Order Books Data.
    Micro[1]: OHLCV Data.



In [35]:
params0= 25, criptos[1], exchanges[0]
params1= 25, criptos[1], exchanges[1]
params2= 25, criptos[1], exchanges[2]
params0,params1,params2

((25, 'ETH/USDT', 'binance'),
 (25, 'ETH/USDT', 'ftx'),
 (25, 'ETH/USDT', 'bytetrade'))

<strong> 
<font color= lightgreen> <font size = 3> Progress check 2.1 </font>

In [40]:
n=10
MS_0 = vs.Micro(params0[0],params0[1], params0[2], n)
MS_1 = vs.Micro(params1[0],params1[1], params1[2], n)
MS_2 = vs.Micro(params2[0],params2[1], params2[2], n)

Bid and Asks Volumes in the Microstructure determine the demand and it's a factor for price movements. This is also true for the VWAP which is a trend detector by reflecting the price adjusted by its volume.

In [41]:
MSob_df = fn.concatenate(MS_0[0], MS_1[0], MS_2[0], n)
MSob_df

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-08T19:18:10.487,25,19.5926,50.9393,70.5319,1339.565,1358.552399
1,binance,2022-11-08T19:18:12.938,25,30.6775,27.9697,58.6472,1339.625,1362.462182
2,binance,2022-11-08T19:18:15.644,25,28.3856,67.8320,96.2176,1338.675,1352.583047
3,binance,2022-11-08T19:18:18.402,25,38.7466,29.9839,68.7305,1337.295,1356.747155
4,binance,2022-11-08T19:18:21.020,25,25.0051,61.3410,86.3461,1337.545,1353.030566
5,binance,2022-11-08T19:18:23.660,25,33.5210,30.0375,63.5585,1338.685,1359.742328
6,binance,2022-11-08T19:18:26.084,25,28.8938,38.5436,67.4374,1339.245,1359.089307
7,binance,2022-11-08T19:18:28.552,25,19.8235,24.6813,44.5048,1340.225,1370.334280
8,binance,2022-11-08T19:18:31.537,25,117.7341,98.2743,216.0084,1340.215,1346.414481
9,binance,2022-11-08T19:18:34.794,25,11.5378,57.1554,68.6932,1340.625,1360.136197


In [42]:
MSohlcv = fn.concatenate(MS_0[1], MS_1[1], MS_2[1], n)
MSohlcv

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-08T19:18:10.487,1337.16,1339.86,1337.15,1339.31,382.8893
1,binance,2022-11-08T19:18:12.938,1337.16,1340.31,1337.15,1339.86,435.4572
2,binance,2022-11-08T19:18:15.644,1337.16,1340.31,1337.15,1338.68,486.4650
3,binance,2022-11-08T19:18:18.402,1337.16,1340.31,1337.15,1337.72,541.0250
4,binance,2022-11-08T19:18:21.020,1337.16,1340.31,1336.92,1337.21,569.7102
5,binance,2022-11-08T19:18:23.660,1337.16,1340.31,1336.92,1338.68,613.7200
6,binance,2022-11-08T19:18:26.084,1337.16,1340.31,1336.92,1339.26,776.5685
7,binance,2022-11-08T19:18:28.552,1337.16,1340.31,1336.92,1340.22,822.4077
8,binance,2022-11-08T19:18:31.537,1337.16,1340.50,1336.92,1340.25,971.5789
9,binance,2022-11-08T19:18:34.794,1337.16,1340.77,1336.92,1340.33,1019.3432


<font color= orange> <font size = 5> 3. Microstructure Modelling: </font>

### <font color= orange> <font size = 5> 4. Bibliography: </font>

<font color= lightgray>

+ Villalobos, O, 2021. **Python Project Template**. Available at: https://github.com/OmarVillalobos/python-project. 
(Accessed: November 5, 2022). 

+ Financial Engineering, **Marketmicrostructure.pptx**. ITESO -  Microstructure and Trading Systems (ITE1731 - DMAF). 
Available at: https://docs.google.com/presentation/d/1uYyMERKKGeZ0MPqbJ3bcir9lfpzbeAH9ZMmxbYysokQ/edit?usp=sharing (Accessed: November 5, 2022). 

+ Muñoz E., J **Roll Model** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/Roll%20.pdf (Accessed: November 5, 2022).

+ Muñoz E., J **Asset Pricing Theory** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/APT.pdf (Accessed: November 5, 2022). 